In [1]:
import os
os.chdir(r"C:\Users\shachini dinushika\Documents\MyDS_Projects")

In [2]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np

In [15]:
def get_title(result):
    try:
        title  = result.find("a", class_= "bookTitle").text.strip()
    except AttributeError:
        title  = ""
    return title

def get_author(result):
    try:
        author = result.find("a", class_= "authorName").text.strip("").replace("  ","")
    except AttributeError:
        author  = ""
    return author

import re
def get_year(result):
    try:
        year = result.find("span", class_= "greyText smallText uitext").text.strip()
        year = re.search(r'\b(\d{4})\b', year)
        if year:
            year=year.group(1)
            year
        else: 0
    except AttributeError:
        year  = ""
    return year

def get_ratings(result):
    try:
        ratings = result.find("span", class_= "minirating").text.strip()
    except AttributeError:
        ratings  = ""
    return ratings


In [16]:
url = 'https://www.goodreads.com/search?utf8=%E2%9C%93&q=best+novels&search_type=books'

In [17]:
# create page scraper function
def page_scrape(url):
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        results = soup.find_all("tr", attrs={'itemtype':'http://schema.org/Book'})
        data =[]
        for result in results:
            title  = get_title(result)
            year  = get_year(result)
            author = get_author(result)
            ratings = get_ratings(result)
            data.append(( title,year, author, ratings))
        return data
    else:
        print(f"Failed to retrieve content. Status code: {r.status_code}")
        return []
    
## create all page scraper function
def all_pages(url, pages):
    all_books_list = [] 
    
    for page in range(1, pages+1):
        page_url = f"{url}&page={page}"
        books_list = page_scrape(page_url)
        all_books_list.extend(books_list) 
    return all_books_list

In [20]:
# Scrape five pages 
url = 'https://www.goodreads.com/search?utf8=%E2%9C%93&q=best+novels&search_type=books'
num_pages_to_scrape = 100  # Set the number of pages to scrape

data = all_pages(url, num_pages_to_scrape)

# Print or do further processing with the 'all_book_titles' list
print(data[1:5])

[('Sandra Brown: Three Complete Novels', '1992', 'Sandra Brown', '4.24 avg rating — 418 ratings'), ('Of Human Bondage (Modern Library 100 Best Novels)', '1915', 'W. Somerset Maugham', '4.14 avg rating — 56,303 ratings'), ('The House of Mirth (Modern Library 100 Best Novels)', '1905', 'Edith Wharton', '3.96 avg rating — 96,286 ratings'), ("Kaidenberg's Best Sons: A Novel in Stories", None, 'Jason Heit', '4.16 avg rating — 19 ratings')]


In [21]:
len(data)

1974

In [22]:
#create a data frame for the books info

books = pd.DataFrame(data,columns =['Title','Year','Author','ratings'])

books.head()

,Title,Year,Author,ratings
0,The Great American Read: The Book of Books: Ex...,2018,Jessica Allen,4.09 avg rating — 959 ratings
1,Sandra Brown: Three Complete Novels,1992,Sandra Brown,4.24 avg rating — 418 ratings
2,Of Human Bondage (Modern Library 100 Best Novels),1915,W. Somerset Maugham,"4.14 avg rating — 56,303 ratings"
3,The House of Mirth (Modern Library 100 Best No...,1905,Edith Wharton,"3.96 avg rating — 96,286 ratings"
4,Kaidenberg's Best Sons: A Novel in Stories,None,Jason Heit,4.16 avg rating — 19 ratings


In [23]:
books.to_csv("goodreads_books.csv",index = False, encoding  = 'utf-8')